In [10]:
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
a = [[1, 2], [2, 2]]
b = [[3, 3], [2, 4]]

In [3]:
print(f'a.dot.b = \n{np.dot(a, b)}')
print(f'b.dot.a = \n{np.dot(b, a)}')

a.dot.b = 
[[ 7 11]
 [10 14]]
b.dot.a = 
[[ 9 12]
 [10 12]]


## simple perceptron

In [4]:
input_data = np.random.random([10, 4])
w = np.random.random([2, 4])
b = np.random.random(2)

In [8]:
p = {}
for i in range(2):
    res = []
    for j in range(10):
        res.append(np.dot(input_data[j], w[i]) + b[i])
    p[fr'p{i}'] = res

p

{'p0': [np.float64(2.0971176231746527),
  np.float64(1.6196309246175509),
  np.float64(1.338319958889088),
  np.float64(1.5110845999390636),
  np.float64(1.3862825106963),
  np.float64(1.9066736078409527),
  np.float64(1.7251045177697653),
  np.float64(1.5192360859022411),
  np.float64(1.9617141619092888),
  np.float64(2.2283885769667564)],
 'p1': [np.float64(1.1657923200926166),
  np.float64(0.7128224428754176),
  np.float64(0.6064119632435353),
  np.float64(0.7606404170524326),
  np.float64(0.6668485738576282),
  np.float64(0.9988806929339737),
  np.float64(0.7943353957911679),
  np.float64(0.7682498964055545),
  np.float64(1.0340448237337265),
  np.float64(1.3169786912973214)]}

## Confusion Matrix

In [13]:
a = [1, 0, 1, 0, 1, 1, 1, 0, 1, 1]
p = [1, 1, 0, 0, 1, 1, 0, 1, 1, 0]

tp, tn, fp, fn = 4, 1, 2, 3

confusion_matrix(a, p)

array([[1, 2],
       [3, 4]])

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score


# --------------------- LOAD & PREPARE DATA ---------------------------
df = pd.read_csv("Datasets/diabetes.csv")

X = df.drop("Outcome", axis=1).values
y = df["Outcome"].values.reshape(-1, 1)

sc = StandardScaler()
X = sc.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# --------------------- ACTIVATIONS ---------------------------
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def dsigmoid(a):
    return a * (1 - a)

def relu(z):
    return np.maximum(0, z)

def drelu(z):
    return (z > 0).astype(float)


# --------------------- NETWORK CONFIG ---------------------------
input_dim = X_train.shape[1]   # 8 features
h1_dim = 16
h2_dim = 8
output_dim = 1

np.random.seed(42)
W1 = np.random.randn(input_dim, h1_dim) * 0.1
b1 = np.zeros((1, h1_dim))

W2 = np.random.randn(h1_dim, h2_dim) * 0.1
b2 = np.zeros((1, h2_dim))

W3 = np.random.randn(h2_dim, output_dim) * 0.1
b3 = np.zeros((1, output_dim))


# --------------------- TRAINING LOOP ---------------------------
lr = 0.01
epochs = 2000
batch_size = 32

for epoch in range(epochs):

    # Shuffle training data
    idx = np.random.permutation(len(X_train))
    X_train = X_train[idx]
    y_train = y_train[idx]

    for i in range(0, len(X_train), batch_size):

        Xb = X_train[i:i+batch_size]
        yb = y_train[i:i+batch_size]

        # -------- Forward --------
        z1 = Xb @ W1 + b1
        a1 = relu(z1)

        z2 = a1 @ W2 + b2
        a2 = relu(z2)

        z3 = a2 @ W3 + b3
        y_pred = sigmoid(z3)

        # -------- Backward --------
        m = len(Xb)
        dz3 = (y_pred - yb)      # derivative for binary cross-entropy

        dW3 = a2.T @ dz3 / m
        db3 = np.mean(dz3, axis=0, keepdims=True)

        dz2 = (dz3 @ W3.T) * drelu(z2)
        dW2 = a1.T @ dz2 / m
        db2 = np.mean(dz2, axis=0, keepdims=True)

        dz1 = (dz2 @ W2.T) * drelu(z1)
        dW1 = Xb.T @ dz1 / m
        db1 = np.mean(dz1, axis=0, keepdims=True)

        # -------- Update weights --------
        W3 -= lr * dW3
        b3 -= lr * db3
        W2 -= lr * dW2
        b2 -= lr * db2
        W1 -= lr * dW1
        b1 -= lr * db1

    # -------- Monitor loss --------
    # if epoch % 200 == 0:
    #     loss = -np.mean(y_train * np.log(y_pred + 1e-8) +
    #                     (1 - y_train) * np.log(1 - y_pred + 1e-8))
    #     print(f"Epoch {epoch}, Loss = {loss:.4f}")


# --------------------- EVALUATION ---------------------------
# Forward on test set
z1 = X_test @ W1 + b1
a1 = relu(z1)

z2 = a1 @ W2 + b2
a2 = relu(z2)

z3 = a2 @ W3 + b3
y_pred_test = sigmoid(z3)

y_pred_labels = (y_pred_test > 0.5).astype(int)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_labels))
print("\nAccuracy:", accuracy_score(y_test, y_pred_labels))

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/diabetes.csv'